In [1]:
using JuMP
using CPLEX
using LinearAlgebra

Now we define the master problem :)

In [2]:
function solve_master(A,b,c, op_A, op_B, feas_A, feas_B)
    nx = size(c, 2)
    m = Model(CPLEX.Optimizer)
    if size(op_A, 1) == 0
        @variable(m, x[1:nx])
        @objective(m, Min, sum(c[i] * x[i] for i = 1:nx))
        @constraint(m, A*x .<= b)
        optimize!(m)
        theta = -Inf
    else
        @variable(m, x[1:nx])
        @variable(m, theta)
        @objective(m, Min, sum(c[i] * x[i] for i = 1:nx) + theta)
        @constraint(m, A*x .<= b)
        for i = 1:size(op_A, 1)
            @constraint(m, sum(op_A[i,j] * x[j] for j = 1:nx) + op_A[i,nx+1] * theta .>= op_B[i])
    
        end
        optimize!(m)
        theta = JuMP.value(theta)
    end
    sol = zeros(1, nx)
    for i = 1:nx
            sol[i] = JuMP.value(x[i])
    end
    sol = vec(sol)
    return (objective_value(m), sol, theta, JuMP.termination_status(m))
end

solve_master (generic function with 1 method)

In [3]:
function step2(W, h, T, x, omega, q)
    m = Model(CPLEX.Optimizer)
    ny = size(W,2)
    
    @variable(m, y[1:ny] >= 0)
    @constraint(m,constraint, W*y - h[:,omega] + T*x .<= 0)
    @objective(m, Min, transpose(q[:,omega]) * y)
    optimize!(m);
    sol = zeros(1, ny)
    for i = 1:ny
        sol[i] = JuMP.value(y[i])
    end
    sol = vec(sol)
    return (objective_value(m), sol, dual.(constraint), JuMP.termination_status(m))
end

step2 (generic function with 1 method)

In [4]:
function main(A, b, c, T, W, h, q, p)
    op_A = []
    op_B = []
    fea_A = []
    fea_B = []
    k = 0
    val_k, x_k, theta_k = 0, 0, 0
    w_old = -Inf
    w_now = -Inf
    while (k<10000000000)
        println(k)
        
        (val_k, x_k, theta_k, termination_status) = solve_master(A, b, c, op_A, op_B, fea_A, fea_B)
    
        if termination_status == MOI.INFEASIBLE
            return (-1, -1, -1)
        end
        
        V_temp = []
        status = 0
        e = 0
        E = zeros(1,size(p,2))
        for w in 1:size(p,2)
            (w_k, y_k, pi_k, termination_status) = step2(W, h, T, x_k, w, q)
            w_now = w_k
            if termination_status == MOI.INFEASIBLE
                S_k = [S_k; sigma_k]
                k += 1
                status = 1
                break
            else
                e += p[w] * transpose(pi_k) * h[:,w]
                E += p[w] * transpose(pi_k) * T
            end
        end
        if(status == 0)
                V_temp = [E 1]
            if op_A == []
                op_A  = V_temp
                op_B  = [e]
                
            else
                op_A = [op_A; V_temp]
                op_B = [op_B;e]
            end
            
            if(w_now <= theta_k)
                return (x_k, y_k, k)
            else
                k += 1
                w_old = w_now
            end
        end
    end
end 

main (generic function with 1 method)

In [5]:
d = [500 100; 300 300]
q = [-24 -28; -28 -32]
p = [0.4 0.6]

A = [1 1; -1 0; 0 -1]
b = [120; -40; -20]
c = [100 150]

T = [-60 0; 0 -80; 0 0; 0 0]
W = [6 10; 8 5; 1 0; 0 1]
h = [0 0; 0 0; d[1,1] d[2,1]; d[1,2] d[2,2]]
(x,y, k) = main(A, b, c, T, W, h, q, p)


0
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 1 time.
LP Presolve eliminated 3 rows and 2 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 1 time.
LP Presolve eliminated 4 rows and 2 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 1 time.
LP Presolve eliminated 2 rows and 0 columns.
Reduced LP has 2 rows, 2 columns, and 4 nonzeros.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =        -10320.000000
1
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 1 time.
LP Presolve eliminated 4 rows and 3 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
Tried aggregator 1 time.
LP Presolve eliminated 4 rows and 

LoadError: UndefVarError: y_k not defined